In [ ]:
!pip install boto3 pyarrow pandas

In [ ]:
import boto3

ACCESS_KEY = "{AWS_ACCESS_KEY_ID}"
SECRET_KEY = "{AWS_SECRET_ACCESS_KEY}"

s3 = boto3.client(
    service_name="s3",
    region_name="kr-central-2",
    endpoint_url="https://objectstorage.kr-central-2.kakaocloud.com",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)

In [ ]:
import io
import random
import pyarrow.parquet as pq
import pandas as pd

bucket_name = "data-catalog-bucket"
prefix      = "data-catalog-dir/user_behavior_prediction_batch/"

resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
keys = [
    obj["Key"]
    for obj in resp.get("Contents", [])
    if obj["Key"].endswith(".parquet")
]

if not keys:
    raise ValueError(f"No parquet files found under prefix {prefix!r}")

In [ ]:
random_key = random.choice(keys)

In [ ]:
obj = s3.get_object(Bucket=bucket_name, Key=random_key)
buf = io.BytesIO(obj["Body"].read())
table = pq.read_table(buf)
df = table.to_pandas()

df.head(10)